In [28]:
 # import
import pandas as pd
import numpy as np

purchase_data = pd.read_csv('purchase_data.csv')
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [29]:
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [30]:
#Display player count

players = len(purchase_data['SN'].unique())
players_df = pd.DataFrame({"Total # of Players": [len(purchase_data.SN.unique())]})
players_df

,Total # of Players
0,576


In [31]:
#Purchasing Analysis (Total)
unique_items = len(purchase_data['Item ID'].unique())
avg_price= purchase_data['Price'].mean()
total_purchases= len(purchase_data['Purchase ID'])

total_revenue= sum(purchase_data['Price'])


summary_df = pd.DataFrame({
    'Unique items': unique_items,
    'Average Price': '${:.2f}'.format(avg_price),
    'Total Purchases': total_purchases,
    'Total Revenue': '${:.2f}'.format(total_revenue)
},index=[0])
summary_df

,Unique items,Average Price,Total Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [32]:
# Gender Demographics

gender = purchase_data.drop_duplicates('SN')

gender_count = gender['Gender'].value_counts()


gender_percent = (gender['Gender'].value_counts()/players)*100

gender_df = pd.DataFrame({
    'Count': gender_count,
    'Percent of Total Players': gender_percent.map('{:.2f}%'.format)
})
gender_df

,Count,Percent of Total Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [33]:
#Purchasing Analysis (Gender)

gender_purchase_data_df = pd.DataFrame({
    'Number of Players': gender_count,
    'Purchase Count': gender_purchase_count,
    'Avg Purchase Price': gender_purchase_mean.map('${:,.2f}'.format),
    'Total Revenue': gender_purchase_total.map('${:,.2f}'.format),
})
gender_purchase_data_df

,Number of Players,Purchase Count,Avg Purchase Price,Total Revenue
Female,81,113,$3.20,$361.94
Male,484,652,$3.02,"$1,967.64"
Other / Non-Disclosed,11,15,$3.35,$50.19


In [34]:
bins = [0, 9, 14, 19, 24, 29, 34, 39, 400]

group_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

In [35]:
purchase_data['age_group'] =  pd.cut(purchase_data['Age'], bins, labels=group_labels)

In [36]:
players_by_age = purchase_data.groupby('age_group').SN.nunique()

age_demo = pd.DataFrame({
    'Total Count': players_by_age,
    'Percentage of Players': (players_by_age / purchase_data.SN.nunique() * 100).map("{:.2f}".format)
})
age_demo

,Total Count,Percentage of Players
age_group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [37]:
# Purchasing Analysis (Age)

purchase_count_age = purchase_data[['age_group', 'Purchase ID']]
purchase_values_age = purchase_data[['age_group', 'Price']]


total_purchase_age = purchase_count_age.groupby('age_group').count() 

total_purchase_values_age = purchase_values_age.groupby('age_group').sum()

purchasing_age_analysis = pd.DataFrame({
    'Purchase Count': total_purchase_age['Purchase ID'],
    'Average Purchase Price': (total_purchase_values_age['Price'] / total_purchase_age['Purchase ID']).map("${:.2f}".format),
    'Total Purchase Value': total_purchase_values_age['Price'].map("${:.2f}".format)
})
purchasing_age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value
age_group,,,
<10,23,$3.35,$77.13
10-14,28,$2.96,$82.78
15-19,136,$3.04,$412.89
20-24,365,$3.05,$1114.06
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


In [41]:
# Top Spenders

spender_purchase = purchase_data["SN"].value_counts()


spender_average_purchase = purchase_data["Price"].groupby(purchase_data["SN"]).mean()

spender_total_purchase = purchase_data["Price"].groupby(purchase_data["SN"]).sum()

big_spenders = pd.DataFrame({
    "Purchase Count":spender_purchase,
    "Average Purchase Price":spender_average_purchase,
    "Total Purchase Value":spender_total_purchase
})


big_spenders_sorted = big_spenders.sort_values("Total Purchase Value", ascending= False)
big_spenders_sorted["Average Purchase Price"] = big_spenders_sorted["Average Purchase Price"].map("${:.2f}".format)
big_spenders_sorted["Total Purchase Value"] = big_spenders_sorted["Total Purchase Value"].map("${:.2f}".format)

big_spenders_sorted[:5]

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [42]:
# Most Popular Items

popular_items = purchase_data[['Item ID', 'Item Name', 'Price']]


items_purchase = popular_items.groupby(["Item ID","Item Name"])["Price"].count()

item_price= popular_items.groupby(["Item ID","Item Name"])["Price"].mean()

popular_purchase_value = popular_items.groupby(["Item ID","Item Name"])["Price"].sum()


popular_items = pd.DataFrame({
    "Purchase Count":items_purchase,
    "Item Price":item_price,
    "Total Purchase Value":popular_purchase_value
})


popular_items_sorted = popular_items.sort_values("Purchase Count", ascending= False)
popular_items_sorted["Item Price"]= popular_items_sorted["Item Price"].map("${:.2f}".format)
popular_items_sorted["Total Purchase Value"]= popular_items_sorted["Total Purchase Value"].map("${:.2f}".format)


popular_items_sorted[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [43]:
 #Most Profitable Items

popular_items_sorted_purchase = popular_items.sort_values("Total Purchase Value", ascending= False)
popular_items_sorted_purchase["Item Price"]= popular_items_sorted_purchase["Item Price"].map("${:.2f}".format)
popular_items_sorted_purchase["Total Purchase Value"]= popular_items_sorted_purchase["Total Purchase Value"].map("${:.2f}".format)

popular_items_sorted_purchase[:5]

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
